In [1]:
import json

In [ ]:
def ParsingJson(textname, filename):
    with open("./data/sample/"+textname+"/"+filename, "r", encoding="utf8") as f:
        contents = f.read()
        chat = json.loads(contents)
    keyJson = []
    if textname=="chat":
        keyJson = "utterances"
    elif textname=="news":
        keyJson = "sentences"
    
    countSentences = len(file[keyJson])
    
    result_list = []
    
    